In [81]:
import pickle
import boto3
import uuid
import botocore
from botocore.exceptions import ClientError
import os, time, json, time
from datetime import datetime

from misc import load_from_yaml, save_to_yaml
# import s3, iam, lftn, glue, lambdafn as lfn, sns, eventbridge as event
# from lambdafn import build_lambda_package, print_latest_lambda_logs

from dotenv import load_dotenv
load_dotenv(os.getenv("MY_AWS_DIR", "") + "/.env")

from aws_log_extractor import LogManager, LogFilter, LogLevel

from mylogger import CustomLogger
logger = CustomLogger()

In [82]:
ACCOUNT_ID        = os.environ['AWS_ACCOUNT_ID_ROOT']
REGION            = os.environ.get('AWS_DEFAULT_REGION', 'us-east-1')
# VPC_ID            = os.environ['AWS_DEFAULT_VPC']
# SECURITY_GROUP_ID = os.environ['AWS_DEFAULT_SG_ID']
# SUBNET_IDS        = SUBNET_IDS = os.environ["AWS_DEFAULT_SUBNET_IDS"].split(":")
# SUBNET_ID         = SUBNET_IDS[0]

logger.info(f"VPC_ID: {ACCOUNT_ID}")

INFO: 2025-10-21 17:40:42 [2366322474.py:8] VPC_ID: 530976901147


In [83]:
s3_client            = boto3.client('s3')
glue_client          = boto3.client('glue')
lakeformation_client = boto3.client('lakeformation')
ec2_resource         = boto3.resource('ec2', region_name=REGION)
events_client        = boto3.client('events')
lambda_client        = boto3.client('lambda')
cw_logs_client       = boto3.client('logs')
logs_client = boto3.client("logs")

#### S3 Bucket Notification Test

In [ ]:
# Your bucket name
bucket_name = "datalake-bkt-10102025"  # replace with var.datalake_bkt.bucket
sales_file_sufix = datetime.now().strftime("%Y%m%d%H%M%S")
sales_file_key = "bronze/sales/sales_tiny.csv"

In [ ]:

# # Put a test object (this should trigger the Lambda)
# response = s3_client.put_object(
#     Bucket=bucket_name, Key="bronze/sales/test_key.txt", Body=json.dumps({"test": "notification"})
# )

# Put a test object (this should trigger the Lambda)
s3_client.upload_file(
    Bucket=bucket_name,
    Key=sales_file_key,
    Filename="/Users/am/mydocs/Software_Development/DEDS/aws/glue/data/sales_tiny1.csv",
)

In [93]:
s3_client.delete_objects(
    Bucket=bucket_name,
    Delete={
        "Objects": [
            {
                "Key": sales_file_key,
            },
        ],
    },
)

{'ResponseMetadata': {'RequestId': 'FF5EJW72MAMEWBYM',
  'HostId': 'b8SE6wVPLmWdYrlAWAvrdOSqBCEaOQnuqA2G0RPS0nQZakImBo9Nh+BR75soPKlPcvFGWmkIFU7bEY+y8adq68BlEEBTumiZYAgzVBv0HCE=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'b8SE6wVPLmWdYrlAWAvrdOSqBCEaOQnuqA2G0RPS0nQZakImBo9Nh+BR75soPKlPcvFGWmkIFU7bEY+y8adq68BlEEBTumiZYAgzVBv0HCE=',
   'x-amz-request-id': 'FF5EJW72MAMEWBYM',
   'date': 'Tue, 21 Oct 2025 23:18:19 GMT',
   'connection': 'close',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Deleted': [{'Key': 'bronze/sales/sales_20251021181101.csv'}]}

In [ ]:
[
    s3_client.delete_object(Bucket=bucket_name, Key=item["Key"])
    # print(item["Key"])
    for item in s3_client.list_objects_v2(Bucket="datalake-bkt-10102025", Prefix="DQE-Results/customers/")[
        "Contents"
    ]
    if item["Key"]
]

#### Extract Lambda logs

In [ ]:
LFN_NAME = "GWF-start-glue-workflow"
log_group_name = f"/aws/lambda/{LFN_NAME}"

In [91]:
! aws logs tail {log_group_name} --since 5m --follow

2025-10-21T23:11:04.860000+00:00 2025/10/21/[$LATEST]c095ece2c0b24e33880f3ae53e06e32b INIT_START Runtime Version: python:3.11.v97	Runtime Version ARN: arn:aws:lambda:us-east-1::runtime:6c687d1ae05d783a82cd99c5896c1baddfe1daf7ac4068082fb989401de8b287
2025-10-21T23:11:05.213000+00:00 2025/10/21/[$LATEST]c095ece2c0b24e33880f3ae53e06e32b [INFO]	2025-10-21T23:11:05.213Z		Found credentials in environment variables.
2025-10-21T23:11:05.376000+00:00 2025/10/21/[$LATEST]c095ece2c0b24e33880f3ae53e06e32b START RequestId: ed7073b1-3b46-4e41-acb1-d626ddf2906b Version: $LATEST
2025-10-21T23:11:05.377000+00:00 2025/10/21/[$LATEST]c095ece2c0b24e33880f3ae53e06e32b {'--BKT_KEY': 'bronze/sales/', '--DATALAKE_BKT': 'datalake-bkt-10102025'}
2025-10-21T23:11:05.378000+00:00 2025/10/21/[$LATEST]c095ece2c0b24e33880f3ae53e06e32b END RequestId: ed7073b1-3b46-4e41-acb1-d626ddf2906b
2025-10-21T23:11:05.378000+00:00 2025/10/21/[$LATEST]c095ece2c0b24e33880f3ae53e06e32b REPORT RequestId: ed7073b1-3b46-4e41-acb1-d626

In [ ]:
log_manager = LogManager(region_name="us-east-1")
lambda_extractor = log_manager.get_lambda_extractor()

# end_time = datetime.now(timezone.utc)
# end_time = datetime.now()
# start_time = end_time - timedelta(hours=8)
# request_id = response["ResponseMetadata"]["RequestId"]

# lambda_logs = lambda_extractor.get_logs_by_time_range(
#     function_name=function_name,
#     start_time=start_time,
#     end_time=end_time,
#     # log_filter=LogFilter(filter_pattern="ERROR"),
#     log_filter=LogFilter(),
# )

lambda_logs = lambda_extractor.get_logs_by_strem_limit(function_name=LFN_NAME)

# lambda_logs = lambda_extractor.get_logs_by_request_id(function_name=LFN_NAME, request_id)

print(lambda_logs)

# Print results
for log in lambda_logs:
    # print(f"{log.timestamp}: {log.level} - {log.message}")
    print(log.message)


#### Test Event Rules

In [72]:
! aws events list-rules --name-prefix GWF-s3-put-rule

{
    "Rules": [
        {
            "Name": "GWF-s3-put-rule",
            "Arn": "arn:aws:events:us-east-1:530976901147:rule/GWF-s3-put-rule",
            "EventPattern": "{\"detail\":{\"object\":{\"key\":[{\"prefix\":\"bronze/sales/\"}]}},\"detail-type\":[\"Object Created\"],\"source\":[\"aws.s3\",\"my-custom-source\"]}",
            "State": "ENABLED",
            "Description": "Triggered when an object is created in the specified S3 bucket.",
            "EventBusName": "default"
        }
    ]
}


In [86]:
! aws events list-targets-by-rule --rule GWF-s3-put-rule

{
    "Targets": [
        {
            "Id": "invoke-lambda",
            "Arn": "arn:aws:lambda:us-east-1:530976901147:function:GWF-start-glue-workflow",
            "Input": "{\"--BKT_KEY\":\"bronze/sales/\",\"--DATALAKE_BKT\":\"datalake-bkt-10102025\"}"
        }
    ]
}


In [ ]:
# ✅ Simulated event structured like a real S3 PutObject event
test_event = {
    "version": "0",
    "id": "abcd-1234-efgh-5678",
    "account": "123456789012",
    "source": "aws.s3",
    "time": "2025-10-12T18:00:00Z",
    "region": "us-east-1",
    "resources": [],
    "detail-type": "Object Created",
    "detail": {
        "object": {"key": "bronze/sales/"},
    },
}


# ✅ Pattern must match exactly this structure
pattern = {
    "source": ["aws.s3"],
    "detail-type": ["Object Created"],
    "detail": {
        "object": {"key": [{"prefix": "bronze/sales/"}]},
    },
}

-   🧠 **Key Facts to Know**:
    -   EventBridge only matches on `detail`, `detail-type`, and `source`
    -   Everything inside detail must exist in the incoming event JSON, in exactly the same structure and data types.

In [76]:

response = events_client.test_event_pattern(
    Event=json.dumps(test_event),
    EventPattern=json.dumps(pattern),
)

print("Event matches pattern:", response["Result"])

Event matches pattern: True


In [79]:
event_detail_data = {
    "object": {"key": "bronze/sales/"},
}

# ✅ Put a custom event to EventBridge
response = events_client.put_events(
    Entries=[
        {
            # Source must be either "aws.s3" or "my-custom-source" as defined in your rule
            "Source": "my-custom-source",
            # DetailType must be "Object Created" as defined in your rule
            "DetailType": "Object Created",
            # The minimal Detail must be a valid JSON string
            "Detail": json.dumps(
                {
                    "object": {"key": "bronze/sales/"},
                }
            ),
            "EventBusName": "default",
        }
    ]
)

print(json.dumps(response, indent=4))

{
    "FailedEntryCount": 0,
    "Entries": [
        {
            "EventId": "1b381b6f-0469-5cb8-7523-4a473e5c3e35"
        }
    ],
    "ResponseMetadata": {
        "RequestId": "7eb90941-d2f5-4b2e-ac7f-7c08d94f91ec",
        "HTTPStatusCode": 200,
        "HTTPHeaders": {
            "x-amzn-requestid": "7eb90941-d2f5-4b2e-ac7f-7c08d94f91ec",
            "content-type": "application/x-amz-json-1.1",
            "content-length": "85",
            "date": "Tue, 21 Oct 2025 22:19:38 GMT"
        },
        "RetryAttempts": 0
    }
}
